<a href="https://colab.research.google.com/github/alexisdr/uned-tfg/blob/main/UNED-TFG-2-data-set-tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del Dataset

Durante este proceso se crea un dataset con lo datos procesados y listos para el entrenamiento del modelo.

# Parametros

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.
* dataset_path: ruta en el que se almacenará el dataset
* labels_file: ruta en la que se almacenará la lista de etiquetas

In [12]:
base_path = '/drive/My Drive/CorpusPFG/'

#Datos de origen
tiny_GoldStandardTrainingEA_data_location = base_path + 'Tiny-GoldStandardTrainingEAs.txt'
training_files_location = base_path + 'Training/'

#Dataset procesado
dataset_path = base_path + 'Tiny-Dataset'
labels_file = dataset_path + '/labels.txt'

In [13]:
!pip install -q datasets

Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [14]:
from google.colab import drive

drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


#Creación del dataset




Procedemos a unir en un dataset etiquetado con los EAs la sección de juicio clínico de los informes que componen el acto clínico. Para esto, del fichero GoldStandardTrainingEAs.txt se toman las líneas en la que los actos clínicos tienen un EA asociado (diferentes de NONE). Por cada fichero que compone el acto clínico se extrae el juicio clínico. El resultado serán 2 columnas:

label: Primer EA
text: Frases del juicio clínico, escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".

Carga del archivo de texto GoldStandardTrainingEAs.txt en un data set

In [17]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", 
                "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(
    tiny_GoldStandardTrainingEA_data_location, sep=' ', header=None, names=column_names)

Procedemos a unir en un dataset etiquetado con los EAs la sección de juicio clínico de los informes que componen el acto clínico.
Para esto, del fichero GoldStandardTrainingEAs.txt se toman las líneas en la que los actos clínicos tienen un EA asociado (diferentes de NONE). Por cada fichero que compone el acto clínico se extrae el juicio clínico. El resultado serán 2 columnas: 
*   label: Primer EA
*   text: Frases del juicio clínico, escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".



Generación de las etiquetas

In [19]:
df_goldStandardTrainingEAs.TieneEA_EA1.unique()

array(['NONE', 'T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1',
       'T81.4XXA', 'T45.1X5A', 'Y83.1'], dtype=object)

In [20]:
import os
from os import path

if (not path.exists(dataset_path)):
  os.mkdir(dataset_path)

In [21]:
from datasets import ClassLabel

def generate_label_list():
  df_labels = df_goldStandardTrainingEAs
   
  label_list = list(df_labels.TieneEA_EA1.unique())
  save_label_list(label_list)
  return ClassLabel(num_classes=len(label_list), names=label_list), label_list

def save_label_list(label_list):
  print(labels_file)
  with open(labels_file,'w') as tfile:
	  tfile.write('\n'.join(label_list))

def read_label_list():
  label_list = [line.strip() for line in open(labels_file, 'r')]
  return ClassLabel(num_classes=len(label_list), names=label_list), label_list

class2label, label_list = generate_label_list()

/drive/My Drive/CorpusPFG/Tiny-Dataset/labels.txt


Precesamiento del texto de los informes

In [23]:
import os

JUICIO_CLINICO = 'Juicio Clínico'
TRATAMIENTO = 'Tratamiento'
FIRMADO_XXX = '_FIRMADO_XXXX_'

TRAINING = 0
VALIDATION = 1

def es_linea_comienzo_seleccion (linea):
  return True #linea.strip() == JUICIO_CLINICO

def es_linea_fin_seleccion (linea):
  return FIRMADO_XXX in linea.strip()

def es_linea_adecuada (linea):
  return linea.strip() != '' #and linea.strip() != JUICIO_CLINICO

def tratar_linea_texto (linea):
  return linea.replace ("- ", "").replace("\n", "").replace('"', '""')

def obtener_texto (acto, label_list, lista_nombres_informes_medicos, files_location):
  #print(f"Acto: {acto} EA: {ea} Archivo: {lista_nombres_informes_medicos}")
  text = ''
  for nombre_informe in lista_nombres_informes_medicos:
    with open (files_location + nombre_informe, 'rt') as informe_medico: 
      seleccionar = False
      for lineaTexto in informe_medico:
        if (es_linea_comienzo_seleccion(lineaTexto)):
          seleccionar = True
        if (es_linea_fin_seleccion(lineaTexto)):
          seleccionar = False
        if (seleccionar and es_linea_adecuada(lineaTexto)):
          text = text + tratar_linea_texto(lineaTexto) + "\n"
      return [acto,
              class2label.str2int(label_list[0].strip()), 
              label_list[0].strip(), 
              label_list, 
              lista_nombres_informes_medicos, text]


def get_label_list(row):
  label_list = list()
  for col in column_names:
    if (col == column_names[0]):
      continue
    if (type(row[col]) == str):
      label_list.append(row[col])
  return label_list

def generate_labeled_array (dataframe, file_list, files_location): 
  dataframe_SinNone = dataframe.reset_index()  
  ds_resultado = []
  for index, row in dataframe_SinNone.iterrows():
    texto_filtro = str(row['Acto']) + '-'
    archivos_acto = filter(lambda x: texto_filtro in x, file_list)       
    ds_resultado.append(obtener_texto(row['Acto'], get_label_list(row), 
                                      list(archivos_acto), files_location))
  return ds_resultado

def save_labeled_dataset (labeled_array):
  ds = pd.DataFrame(labeled_array, columns=['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'])
  #ds.to_csv(labeled_dataset_name)
  return ds

def generate_labeled_arrays ():
  training_files = [f for f in os.listdir(training_files_location) 
                      if os.path.isfile(os.path.join(training_files_location, f))]
  labeled_array_training = generate_labeled_array (
      df_goldStandardTrainingEAs, training_files, training_files_location)
  ds_labeled_training = save_labeled_dataset (labeled_array_training)

  return ds_labeled_training

datasets = generate_labeled_arrays()

Composición del dataset

In [24]:
from datasets import Dataset, Value, DatasetDict

train_dataset = Dataset.from_pandas(datasets)

def set_dataset_features (dataset):
  new_features = dataset.features.copy()
  new_features["label"] = class2label
  return dataset.cast(new_features)
  
train_dataset = set_dataset_features (train_dataset)
train_val_dataset = train_dataset.train_test_split(test_size=0.1)

dataset_complete = DatasetDict(
    {
        "train": train_val_dataset["train"],
        "validation": train_val_dataset["test"]
    })

dataset_complete

Casting the dataset:   0%|          | 0/811 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 729
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 82
    })
})

In [25]:
dataset_complete["train"][4]

{'acto': 24866006,
 'label': 0,
 'label_str': 'NONE',
 'labels_str': ['NONE'],
 'informes': ['24866006-163505016.txt'],
 'text': ' _CABECERA_XXXX_  \nSexo :Hombre\n\tINFORME ALTA DE CIRUGÍA GENERAL Y DEL APARATO DIGESTIVO\n\tNIF/ _DATA_XXXX_ \n\t _DATA_XXXX_ \n\t _DATA_XXXX_ \n\t _DATA_XXXX_ \n\t_NOMBRE_XXXX_  \n\tTeléfono:  _DATA_XXXX_ \n\tTeléfono Móvil:  _DATA_XXXX_ \n\tFecha ingreso:  _FECHA_XXXX_   _HORA_XXXX_ \nFecha alta:        _FECHA_XXXX_ \n\tTipo de Ingreso: Desde urgencias\n\tMotivo de Ingreso COLECISTITIS AGUDA\nMotivo de Alta\n _APELLIDO_XXXX_ \nVarón de 65 años, sin alergias medicamentosas conocidas.\nDM tipo 2 en tratamiento con Insulina+ADO. HTA. Dislipemia.\nIngreso en Octure/2014 por ITU febril y deterioro de función renal por E.Coli multirresistente, tratada con ertapenem. Hernia discal L4-L5.\nCirugía bariátrica por obesidad mórbida hace 1 año: by-pass laparoscòpico con pié de asa a 150 cm; con pérdida de 27 kg de peso.\nTratamiento actual: Tresiba 44-0-0. Metformi

In [26]:
dataset_complete.save_to_disk(dataset_path)

Flattening the indices:   0%|          | 0/729 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/729 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/82 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/82 [00:00<?, ? examples/s]